In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [2]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [4]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [5]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    items = items_weights['item_id'].values
    weights = items_weights['weights'].values
    recs = np.random.choice(items, size=n, replace=False, p=weights)
    
    return recs.tolist()

In [6]:
%%time

items_weights = data.groupby('item_id')['sales_value'].sum().reset_index()
items_weights['weights'] = items_weights['sales_value'] / items_weights['sales_value'].sum()
items_weights = items_weights.drop(['sales_value'], axis=1)

result['weighted_random_recommendation'] = result['user_id'].apply(
    lambda x: weighted_random_recommendation(items_weights, n=5)
)
result.head(2)

Wall time: 1.77 s


,user_id,actual,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[15831364, 5571328, 12984929, 6463649, 10281773]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1136486, 1004906, 12582207, 849202, 1000050]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [7]:
result_web = pd.read_csv('predictions_basic.csv')
result_web.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[864243, 9835408, 1184894, 937679, 882933]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[1008765, 991870, 2534864, 849701, 1056106]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [8]:
def precision_at_k(recommended_list, bought_list, k=5):
    recommended_list = recommended_list[:k]
    flags = np.isin(recommended_list, bought_list)
    precision = flags.sum() / len(recommended_list)
    return precision

In [9]:
cols = result_web.columns

In [10]:
for col in cols[2:]:
    result_web[f'precision_{col}'] = result_web['user_id'].apply(
    lambda x: precision_at_k(result_web[col].values, result_web['actual'].values)
    )

In [11]:
result_web.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,precision_random_recommendation,precision_popular_recommendation,precision_itemitem,precision_cosine,precision_tfidf,precision_own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[864243, 9835408, 1184894, 937679, 882933]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]",0.0,0.0,0.0,0.0,0.0,0.0
1,3,[ 835476 851057 872021 878302 879948 ...,"[1008765, 991870, 2534864, 849701, 1056106]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]",0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
result_web[-6:].describe()

,user_id,precision_random_recommendation,precision_popular_recommendation,precision_itemitem,precision_cosine,precision_tfidf,precision_own_purchases
count,6.000000,6.0,6.0,6.0,6.0,6.0,6.0
mean,2497.500000,0.0,0.0,0.0,0.0,0.0,0.0
std,1.870829,0.0,0.0,0.0,0.0,0.0,0.0
min,2495.000000,0.0,0.0,0.0,0.0,0.0,0.0
25%,2496.250000,0.0,0.0,0.0,0.0,0.0,0.0
50%,2497.500000,0.0,0.0,0.0,0.0,0.0,0.0
75%,2498.750000,0.0,0.0,0.0,0.0,0.0,0.0
max,2500.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
result_web[-6:].mean()

user_id                             2497.5
precision_random_recommendation        0.0
precision_popular_recommendation       0.0
precision_itemitem                     0.0
precision_cosine                       0.0
precision_tfidf                        0.0
precision_own_purchases                0.0
dtype: float64

### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [14]:
result_new = data_test.groupby('user_id')['item_id'].unique().reset_index()
result_new.columns=['user_id', 'actual']

## 1.1 Random recommendation at top-5000

In [15]:
top_5000 = data.groupby('item_id')['sales_value'].sum().reset_index()
top_5000.sort_values('sales_value', ascending=False, inplace=True)
items = top_5000.head(5000).item_id

In [16]:
def random_recommendation_at_top_5000(items, n=5):
    """Случайные рекоммендации из топ-5000 товаров"""

    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [17]:
%%time

result_new['rr_5_at_top_5000'] = result_new['user_id'].apply(
    lambda x: random_recommendation_at_top_5000(items, n=5)
)
result_new['pres_rr_5'] = result_new['user_id'].apply(
    lambda x: precision_at_k(result_new['rr_5_at_top_5000'], result_new['actual'], k=5)
)

Wall time: 57.6 s


In [18]:
result_new['pres_rr_5'].mean()

0.0

*Увеличим список рекомендованных товаров*

In [19]:
%%time

result_new['rr_50_at_top_5000'] = result_new['user_id'].apply(
    lambda x: random_recommendation_at_top_5000(items, n=50)
)
result_new['pres_rr_50'] = result_new['user_id'].apply(
    lambda x: precision_at_k(result_new['rr_50_at_top_5000'], result_new['actual'], k=50)
)

Wall time: 51.6 s


In [20]:
result_new['pres_rr_50'].mean()

0.0

## 1.2 Popularity-based recommendation at top-5000

In [21]:
def popularity_recommendation(items, n=5):
    """Топ-n популярных товаров"""
    
    return items[:n].tolist()

In [22]:
result_new['popular_recommendation'] = result_new['user_id'].apply(lambda x: popularity_recommendation(items, n=5))

result_new['pres_pr_5'] = result_new['user_id'].apply(
    lambda x: precision_at_k(result_new['popular_recommendation'], result_new['actual'], k=5)
)

In [23]:
result_new['pres_pr_5'].mean()

0.0

In [24]:
result_new['popular_recommendation_500'] = result_new['user_id'].apply(lambda x: popularity_recommendation(items, n=500))

result_new['pres_pr_500'] = result_new['user_id'].apply(
    lambda x: precision_at_k(result_new['popular_recommendation_500'], result_new['actual'], k=500)
)

In [25]:
result_new['pres_pr_500'].mean()

0.0